In [ ]:
%reload_ext autoreload
%autoreload 2
# %matplotlib notebook+

In [ ]:
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.visualization
import named_arrays as na
import esis

In [ ]:
astropy.visualization.quantity_support();

In [ ]:
level_0 = esis.flights.f1.data.level_0()

In [ ]:
time = level_0.inputs.time
time = time.replace(ndarray=time.ndarray.datetime)

In [ ]:
level_1 = level_0.unbiased

In [ ]:
fig, ax = plt.subplots(
    figsize=(6, 2.5),
    constrained_layout=True,
)
na.plt.plot(
    time,
    level_1.outputs.mean((level_1.axis_x, level_1.axis_y)),
    axis=level_1.axis_time,
    label=level_1.channel,
    drawstyle='steps-mid',
)
ax.axvspan(
    xmin=level_1.lights.inputs.time_start.ndarray.min().datetime,
    xmax=level_1.lights.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    label="lights",
)
ax.axvspan(
    xmin=level_1.darks_up.inputs.time_start.ndarray.min().datetime,
    xmax=level_1.darks_up.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    label="darks",
    color="gray",
)
ax.axvspan(
    xmin=level_1.darks_down.inputs.time_start.ndarray.min().datetime,
    xmax=level_1.darks_down.inputs.time_end.ndarray.max().datetime,
    alpha=0.3,
    color="gray",
)
ax.set_xlabel("time (UTC)")
ax.set_ylabel("mean intensity (DN)")
ax.legend();

In [ ]:
level_1.darks.to_jshtml()

In [ ]:
taps = level_1.darks.taps

In [ ]:
axes_tap_xy = (taps.axis_tap_x, taps.axis_tap_y)
axis_tap_xy = "tap_xy"
taps_flat = taps.combine_axes(
    axes=axes_tap_xy,
    axis_new=axis_tap_xy
)

fig, ax = na.plt.subplots(
    axis_rows=level_0.axis_channel,
    axis_cols=axis_tap_xy,
    nrows=taps_flat.shape[level_0.axis_channel],
    ncols=taps_flat.shape[axis_tap_xy],
    sharex=True,
    sharey=True,
    constrained_layout=True,
    origin="upper",
)
na.plt.plot(
    taps_flat.outputs.mean_trimmed(.01, (level_0.axis_time, level_0.axis_y)),
    axis=level_0.axis_x,
    ax=ax,
)
na.plt.set_ylim(
    bottom=-1,
    top=1,
    ax=ax,
)
na.plt.axvspan(
    xmin=0,
    xmax=taps.camera.sensor.num_blank,
    color="green",
    alpha=0.2,
    ax=ax,
    label="blank",
)
na.plt.axvspan(
    xmin=taps.num_x - taps.camera.sensor.num_overscan,
    xmax=taps.num_x,
    color="red",
    alpha=0.2,
    ax=ax,
    label="overscan",
)
na.plt.set_xlabel("columns", ax=ax[{level_0.axis_channel: ~0}])
na.plt.text(
    x=0.5,
    y=1.02,
    s=taps.label[{level_0.axis_channel: 0}].combine_axes(axes_tap_xy, axis_tap_xy),
    ax=ax[{level_0.axis_channel: 0}],
    transform=na.plt.transAxes(ax[{level_0.axis_channel: 0}]),
    ha="center",
    va="bottom",
)
na.plt.text(
    x=1.02,
    y=0.5,
    s=level_0.channel,
    ax=ax[{axis_tap_xy: ~0}],
    transform=na.plt.transAxes(ax[{axis_tap_xy: ~0}]),
    ha="left",
    va="center",
);
# ax.ndarray.flat[~0].legend();

In [ ]:
level_1.shape

In [ ]:
level_1 = level_1.active
level_1.shape

In [ ]:
dark = level_1.dark

In [ ]:
fig, ax = na.plt.subplots(
    axis_rows=dark.axis_channel,
    nrows=dark.shape[dark.axis_channel],
    sharex=True,
    sharey=True,
    constrained_layout=True,
    figsize=(5, 8),
    origin="upper",
)
na.plt.set_xlabel("detector $x$ (pix)", ax=ax[{dark.axis_channel: ~0}])
na.plt.set_ylabel("detector $y$ (pix)", ax=ax)

norm = plt.Normalize(-2, 2)

colorizer = plt.Colorizer(norm=norm)

i = {dark.axis_time: 0}

ani = na.plt.pcolormesh(
    dark[i].inputs.pixel.x,
    dark[i].inputs.pixel.y,
    C=dark[i].outputs,
    ax=ax,
    colorizer=colorizer,
)
na.plt.text(
    x=0.5,
    y=1.01,
    s=dark[i].channel,
    transform=na.plt.transAxes(ax),
    ax=ax,
    ha="center",
    va="bottom",
)
na.plt.set_aspect("equal", ax=ax)

plt.colorbar(
    mappable=plt.cm.ScalarMappable(colorizer=colorizer),
    ax=ax.ndarray,
    label=f"signal ({dark.outputs.unit:latex_inline})",
);